### Create single table:

In [1]:
import sqlite3
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
conn = sqlite3.connect(r"D:\Coding\Data-Prep-EDA\Code\Data\online_shop.db")
transactions = pd.read_sql("SELECT * FROM TRANSACTIONS", conn)
transactions.head()

,customer,item_id,purchase_date
0,Ava,1011,4/1/23
1,Ava,1014,4/1/23
2,Ava,1015,4/15/23
3,Ava,1018,5/1/23
4,Ben,2345,4/15/23


In [3]:
items = pd.read_sql("SELECT * FROM ITEMS", conn)
items.head()

,item_id,item_description,price,category,rating
0,1011,Paint,$15.99,Arts & Crafts,3.5
1,1012,Crayons,$2.87,Arts & Crafts,4.7
2,1013,Markers,$2.50,Arts & Crafts,4.8
3,1014,Brush,$1.99,Arts & Crafts,4.2
4,1015,Paper,$22.49,Arts & Crafts,4.5


In [4]:
df = transactions.merge(items, how="left", on="item_id")
df.head()

,customer,item_id,purchase_date,item_description,price,category,rating
0,Ava,1011,4/1/23,Paint,$15.99,Arts & Crafts,3.5
1,Ava,1014,4/1/23,Brush,$1.99,Arts & Crafts,4.2
2,Ava,1015,4/15/23,Paper,$22.49,Arts & Crafts,4.5
3,Ava,1018,5/1/23,Scissors,$3.50,Arts & Crafts,4.6
4,Ben,2345,4/15/23,Dog Food,$29.99,Pet Supplies,4.9


In [5]:
df.shape

(110, 7)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customer          110 non-null    object
 1   item_id           110 non-null    int64 
 2   purchase_date     110 non-null    object
 3   item_description  110 non-null    object
 4   price             110 non-null    object
 5   category          110 non-null    object
 6   rating            110 non-null    object
dtypes: int64(1), object(6)
memory usage: 6.1+ KB


### Prepare rows for modelling:

In [7]:
df.dtypes

customer            object
item_id              int64
purchase_date       object
item_description    object
price               object
category            object
rating              object
dtype: object

In [8]:
df.purchase_date = pd.to_datetime(df.purchase_date)
df.price = pd.to_numeric(df.price.str.replace("$", ""))
df.rating = pd.to_numeric(df.rating)

C:\Users\HP\AppData\Local\Temp\ipykernel_228\1581114149.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.purchase_date = pd.to_datetime(df.purchase_date)


In [9]:
df.dtypes

customer                    object
item_id                      int64
purchase_date       datetime64[ns]
item_description            object
price                      float64
category                    object
rating                     float64
dtype: object

In [10]:
df_apr_may = df[df.purchase_date.dt.month < 6]
df_apr_may.head()

,customer,item_id,purchase_date,item_description,price,category,rating
0,Ava,1011,2023-04-01,Paint,15.99,Arts & Crafts,3.5
1,Ava,1014,2023-04-01,Brush,1.99,Arts & Crafts,4.2
2,Ava,1015,2023-04-15,Paper,22.49,Arts & Crafts,4.5
3,Ava,1018,2023-05-01,Scissors,3.50,Arts & Crafts,4.6
4,Ben,2345,2023-04-15,Dog Food,29.99,Pet Supplies,4.9


In [11]:
df_apr_may.shape

(93, 7)

In [12]:
df_june = df[df.purchase_date.dt.month == 6]
df_june.head()

,customer,item_id,purchase_date,item_description,price,category,rating
6,Ben,2345,2023-06-15,Dog Food,29.99,Pet Supplies,4.9
12,Chloe,2345,2023-06-06,Dog Food,29.99,Pet Supplies,4.9
24,Blake,2545,2023-06-10,Cat Food,25.55,Pet Supplies,4.2
34,Maxwell,5674,2023-06-02,Video Game,30.99,Games & Toys,4.1
36,Isabel,3811,2023-06-15,Socks,7.50,Apparel,3.7


In [13]:
df_june.shape

(17, 7)

In [15]:
dog_food_rows = df_june[df_june.item_description == "Dog Food"]
dog_food_rows.head()

,customer,item_id,purchase_date,item_description,price,category,rating
6,Ben,2345,2023-06-15,Dog Food,29.99,Pet Supplies,4.9
12,Chloe,2345,2023-06-06,Dog Food,29.99,Pet Supplies,4.9
41,Olivia,2345,2023-06-01,Dog Food,29.99,Pet Supplies,4.9
57,Aiden,2345,2023-06-04,Dog Food,29.99,Pet Supplies,4.9
70,Lia,2345,2023-06-20,Dog Food,29.99,Pet Supplies,4.9


In [17]:
june_dog_food_purchases = dog_food_rows.groupby('customer')['item_id'].count().rename('june_dog_food_purchases')
june_dog_food_purchases.head()

customer
Aiden     1
Ben       1
Calvin    1
Chloe     1
Lia       1
Name: june_dog_food_purchases, dtype: int64

In [18]:
total_spend = df_apr_may.groupby('customer')['price'].sum().rename('total_spend')
total_spend.head()

customer
Aiden      222.16
Ava         43.97
Ben         44.19
Bennett     27.73
Blake       25.55
Name: total_spend, dtype: float64

In [20]:
model_df = pd.concat([june_dog_food_purchases, total_spend], axis=1).fillna(0).reset_index().rename(columns={'index':'customer'})
model_df.head()

,customer,june_dog_food_purchases,total_spend
0,Aiden,1.0,222.16
1,Ben,1.0,44.19
2,Calvin,1.0,29.99
3,Chloe,1.0,36.33
4,Lia,1.0,78.95


In [21]:
model_df.tail()

,customer,june_dog_food_purchases,total_spend
17,Madeline,0.0,122.63
18,Margaret,0.0,7.99
19,Maxwell,0.0,78.31
20,Nolan,0.0,67.51
21,Sophie,0.0,2.57


### Dummy Variables:

In [22]:
model_df.head()

,customer,june_dog_food_purchases,total_spend
0,Aiden,1.0,222.16
1,Ben,1.0,44.19
2,Calvin,1.0,29.99
3,Chloe,1.0,36.33
4,Lia,1.0,78.95


In [27]:
category_dummies = pd.get_dummies(df_apr_may.category)
category_dummies.head()

,Apparel,Arts & Crafts,Games & Toys,Personal Care,Pet Supplies
0,False,True,False,False,False
1,False,True,False,False,False
2,False,True,False,False,False
3,False,True,False,False,False
4,False,False,False,False,True


In [28]:
category_dummies = pd.get_dummies(df_apr_may.category, dtype=int)
category_dummies.head()

,Apparel,Arts & Crafts,Games & Toys,Personal Care,Pet Supplies
0,0,1,0,0,0
1,0,1,0,0,0
2,0,1,0,0,0
3,0,1,0,0,0
4,0,0,0,0,1


In [29]:
pd.concat([df_apr_may.customer, category_dummies], axis=1)

,customer,Apparel,Arts & Crafts,Games & Toys,Personal Care,Pet Supplies
0,Ava,0,1,0,0,0
1,Ava,0,1,0,0,0
2,Ava,0,1,0,0,0
3,Ava,0,1,0,0,0
4,Ben,0,0,0,0,1
...,...,...,...,...,...,...
105,Jenny,0,1,0,0,0
106,Jenny,0,1,0,0,0
107,Jenny,0,1,0,0,0
108,Jenny,0,1,0,0,0


In [31]:
categories = pd.concat([df_apr_may.customer, category_dummies], axis=1).groupby('customer').sum().reset_index()
categories.head()

,customer,Apparel,Arts & Crafts,Games & Toys,Personal Care,Pet Supplies
0,Aiden,0,0,0,0,8
1,Ava,0,4,0,0,0
2,Ben,0,0,0,0,2
3,Bennett,0,5,0,0,0
4,Blake,0,0,0,0,1


In [32]:
model_df = model_df.merge(categories, how='left', on='customer')
model_df.head()

,customer,june_dog_food_purchases,total_spend,Apparel,Arts & Crafts,Games & Toys,Personal Care,Pet Supplies
0,Aiden,1.0,222.16,0,0,0,0,8
1,Ben,1.0,44.19,0,0,0,0,2
2,Calvin,1.0,29.99,0,0,0,0,1
3,Chloe,1.0,36.33,3,2,0,0,0
4,Lia,1.0,78.95,2,0,0,0,1
